<a href="https://colab.research.google.com/github/fasterinnerlooper/Notebooks/blob/main/training/training_with_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [7]:
%pip install tensorflow==2.14 intel-extension-for-tensorflow[xpu]==2.14 intel-optimization-for-horovod==0.28.1.1 torch==2.0.1 torchvision==0.15.2 intel_extension_for_pytorch==2.0.100 oneccl-bind-pt==2.0.0 --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/cpu/us/ -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement intel-extension-for-tensorflow==2.14 (from versions: 0.0.0.dev1)
ERROR: No matching distribution found for intel-extension-for-tensorflow==2.14


In [8]:
%pip install wandb -q

Note: you may need to restart the kernel to use updated packages.


In [9]:
import wandb

wandb.init(project="codeBERTa-on-csharp")

In [10]:
%pip install numpy transformers datasets transformers[torch] accelerate -U -q



Note: you may need to restart the kernel to use updated packages.


In [11]:
# This script needs these libraries to be installed:
#   numpy, transformers, datasets

import wandb

import os
import numpy as np
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

model_name = "huggingface/CodeBERTa-small-v1"

# download prepare the data
# dataset = load_dataset("code_search_net","javascript")
dataset = load_dataset("microsoft/LCC_csharp")

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [13]:
dataset = dataset.map(
    lambda x: tokenizer(x["context"], truncation=True, padding="max_length"),
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/100000 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42)
small_eval_dataset = dataset["test"].shuffle(seed=42)

block_size = 250


def group_texts(examples):
    # Concatenate all texts.
    # concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    concatenated_examples = examples
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


small_train_dataset = small_train_dataset.map(
    group_texts, truncation=True, batched=True, num_proc=4
)
small_eval_dataset = small_eval_dataset.map(
    group_texts, truncation=True, batched=True, num_proc=4
)

In [ ]:
# download the model
model = AutoModelForMaskedLM.from_pretrained(model_name)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"] = "training-with-wandb"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"] = "true"

# turn off watch to log faster
os.environ["WANDB_WATCH"] = "false"

# pass "wandb" to the 'report_to' parameter to turn on wandb logging
training_args = TrainingArguments(
    output_dir="models",
    report_to="wandb",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
import math

wandb.init(project="codeBERTa-on-csharp")

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")